# Пример за използване на инструмента AutoGen


## Импортиране на необходимите пакети


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Създаване на клиента

В този пример ще използваме [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) за достъп до LLM.

`model` е дефиниран като `gpt-4o-mini`. Опитайте да промените модела на друг, наличен в GitHub Models marketplace, за да видите различни резултати.

Като бърз тест, просто ще изпълним прост въпрос - `Каква е столицата на Франция`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Дефиниране на функциите

В този пример ще предоставим на агента достъп до инструмент, който представлява функция със списък на наличните ваканционни дестинации и тяхната наличност.

Можете да си представите, че това би било сценарий, в който туристически агент има достъп до туристическа база данни, например.

Докато разглеждате този пример, не се колебайте да опитате да дефинирате нови функции и инструменти, които агентът може да извиква.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Определяне на инструмента за функция
За да може агентът да използва `vacation_destinations` като `FunctionTool`, трябва да го дефинираме като такъв.

Също така ще предоставим описание на инструмента, което е полезно за агента да разбере за какво се използва този инструмент във връзка със задачата, която потребителят е поискал.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Определяне на агента

Сега можем да създадем агента в кода по-долу. Определяме `system_message`, за да дадем на агента инструкции как да помага на потребителите да намират дестинации за почивка.

Също така задаваме параметъра `reflect_on_tool_use` на true. Това позволява на LLM да вземе отговора от извикването на инструмента и да го изпрати, използвайки естествен език.

Можете да зададете параметъра на false, за да видите разликата.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Стартиране на агента

Сега можем да стартираме агента с първоначалното съобщение от потребителя, който иска да направи пътуване до Токио.

Можете да промените тази дестинация, за да видите как агента реагира на наличността на града.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
